In [28]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('ecommerce.db')


In [29]:

# 1. Retrieve the top 5 customers who have made the highest average order amounts in the last 6 months. The average order amount should be calculated for each customer, and the result should be sorted in descending order.

cursor = conn.cursor()
cursor.execute('''
    SELECT Customer.CustomerID, Customer.Name, AVG("Order".OrderAmount) AS average_order_amount
    FROM Customer
    JOIN "Order" ON Customer.CustomerID = "Order".CustomerID
    WHERE "Order".OrderDate >= DATE('now', '-6 months')
    GROUP BY Customer.CustomerID, Customer.Name
    ORDER BY average_order_amount DESC
    LIMIT 5;
    ''')

query = cursor.fetchall()
print(query)

[(9, 'Grace Harris', 641.9100000000001), (6, 'Diana Evans', 556.6700000000001), (4, 'Bob Brown', 485.41), (7, 'Eve Foster', 413.3533333333334), (1, 'John Doe', 266.29333333333335)]


In [30]:
# 2. Retrieve the list of customer whose order value is lower this year as compared to previous year

cursor = conn.cursor()
cursor.execute('''
    SELECT c.CustomerID, c.Name, 
       COALESCE(CurrentYear.CurrentYearTotal, 0) AS CurrentYearTotal, 
       COALESCE(PreviousYear.PreviousYearTotal, 0) AS PreviousYearTotal
    FROM customer c
    LEFT JOIN (
        SELECT o.CustomerID, SUM(o.OrderAmount) AS CurrentYearTotal
        FROM "Order" o
        WHERE strftime('%Y', o.OrderDate) = strftime('%Y', 'now')
        GROUP BY o.CustomerID
    ) AS CurrentYear ON c.CustomerID = CurrentYear.CustomerID
    LEFT JOIN (
        SELECT o.CustomerID, SUM(o.OrderAmount) AS PreviousYearTotal
        FROM "Order" o
        WHERE strftime('%Y', o.OrderDate) = strftime('%Y', 'now', '-1 year')
        GROUP BY o.CustomerID
    ) AS PreviousYear ON c.CustomerID = PreviousYear.CustomerID
    WHERE COALESCE(CurrentYear.CurrentYearTotal, 0) < COALESCE(PreviousYear.PreviousYearTotal, 0);
    ''')

query = cursor.fetchall()
print(query)

[(2, 'Jane Smith', 0, 652.52), (3, 'Alice Johnson', 0, 1281.46), (5, 'Charlie Davis', 213.57, 1067.75), (8, 'Frank Green', 0, 762.3500000000001), (9, 'Grace Harris', 641.9100000000001, 789.18)]


In [31]:
# 3. Create a table showing cumulative purchase by a particular customer. Show the breakup of cumulative purchases by product category

cursor = conn.cursor()
cursor.execute('''
    SELECT 
        c.CustomerID, 
        c.Name, 
        p.Category, 
        SUM(oi.Quantity * oi.PriceAtPurchase) AS TotalPurchase
    FROM 
        Customer c
    JOIN 
        "Order" o ON c.CustomerID = o.CustomerID
    JOIN 
        OrderItem oi ON o.OrderID = oi.OrderID
    JOIN 
        Variant v ON oi.VariantID = v.VariantID
    JOIN 
        Product p ON v.ProductID = p.ProductID
    GROUP BY 
        c.CustomerID, 
        c.Name, 
        p.Category
    ORDER BY 
        c.CustomerID, 
        p.Category;
    ''')

query = cursor.fetchall()
print(query)

[(1, 'John Doe', 'Clothing', 419.13), (1, 'John Doe', 'Electronics', 581.56), (1, 'John Doe', 'Groceries', 98.44), (2, 'Jane Smith', 'Clothing', 301.09000000000003), (2, 'Jane Smith', 'Electronics', 320.8), (2, 'Jane Smith', 'Groceries', 30.630000000000003), (3, 'Alice Johnson', 'Clothing', 924.12), (3, 'Alice Johnson', 'Electronics', 482.35), (3, 'Alice Johnson', 'Groceries', 129.07), (4, 'Bob Brown', 'Clothing', 512.03), (4, 'Bob Brown', 'Electronics', 586.98), (4, 'Bob Brown', 'Groceries', 60.040000000000006), (5, 'Charlie Davis', 'Clothing', 302.42), (5, 'Charlie Davis', 'Electronics', 860.04), (5, 'Charlie Davis', 'Groceries', 118.86000000000001), (6, 'Diana Evans', 'Clothing', 475.98), (6, 'Diana Evans', 'Electronics', 725.64), (6, 'Diana Evans', 'Groceries', 29.41), (7, 'Eve Foster', 'Clothing', 1172.38), (7, 'Eve Foster', 'Electronics', 270.91), (7, 'Eve Foster', 'Groceries', 227.51000000000002), (8, 'Frank Green', 'Clothing', 826.7800000000001), (8, 'Frank Green', 'Electronics

In [32]:
# 4. Retrieve the list of top 5 selling products. Further bifurcate the sales by product variants

conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

cursor.execute('''
    SELECT 
        p.Name AS ProductName , 
        v.Name AS VariantName, 
        SUM(oi.Quantity) AS TotalQuantitySold
    FROM 
        OrderItem oi
    JOIN 
        Variant v ON oi.VariantID = v.VariantID
    JOIN 
        Product p ON v.ProductID = p.ProductID
    GROUP BY 
        p.Name, 
        v.Name
    ORDER BY 
        SUM(oi.Quantity) DESC
    LIMIT 5;
    ''')

query = cursor.fetchall()
print(query)

[('Phone', 'Smartphone', 40), ('Jeans', 'Blue Jeans', 39), ('Headphones', 'Wired Headphones', 33), ('Jacket', 'Leather Jacket', 33), ('Milk', 'Whole Milk', 27)]


In [33]:
conn.close()